In [1]:

# imports
import os
import sys
import types
import json
import base64

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96
interactivity = ''
is_shiny = False
is_dashboard = False
plotly_connected = True

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = "figure"
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  if plotly_connected:
    pio.renderers.default = "notebook_connected"
  else:
    pio.renderers.default = "notebook"
  for template in pio.templates.keys():
    pio.templates[template].layout.margin = dict(t=30,r=0,b=0,l=0)
except Exception:
  pass

# disable itables paging for dashboards
if is_dashboard:
  try:
    from itables import options
    options.dom = 'fiBrtlp'
    options.maxBytes = 1024 * 1024
    options.language = dict(info = "Showing _TOTAL_ entries")
    options.classes = "display nowrap compact"
    options.paging = False
    options.searching = True
    options.ordering = True
    options.info = True
    options.lengthChange = False
    options.autoWidth = False
    options.responsive = True
    options.keys = True
    options.buttons = []
  except Exception:
    pass
  
  try:
    import altair as alt
    # By default, dashboards will have container sized
    # vega visualizations which allows them to flow reasonably
    theme_sentinel = '_quarto-dashboard-internal'
    def make_theme(name):
        nonTheme = alt.themes._plugins[name]    
        def patch_theme(*args, **kwargs):
            existingTheme = nonTheme()
            if 'height' not in existingTheme:
              existingTheme['height'] = 'container'
            if 'width' not in existingTheme:
              existingTheme['width'] = 'container'

            if 'config' not in existingTheme:
              existingTheme['config'] = dict()
            
            # Configure the default font sizes
            title_font_size = 15
            header_font_size = 13
            axis_font_size = 12
            legend_font_size = 12
            mark_font_size = 12
            tooltip = False

            config = existingTheme['config']

            # The Axis
            if 'axis' not in config:
              config['axis'] = dict()
            axis = config['axis']
            if 'labelFontSize' not in axis:
              axis['labelFontSize'] = axis_font_size
            if 'titleFontSize' not in axis:
              axis['titleFontSize'] = axis_font_size  

            # The legend
            if 'legend' not in config:
              config['legend'] = dict()
            legend = config['legend']
            if 'labelFontSize' not in legend:
              legend['labelFontSize'] = legend_font_size
            if 'titleFontSize' not in legend:
              legend['titleFontSize'] = legend_font_size  

            # The header
            if 'header' not in config:
              config['header'] = dict()
            header = config['header']
            if 'labelFontSize' not in header:
              header['labelFontSize'] = header_font_size
            if 'titleFontSize' not in header:
              header['titleFontSize'] = header_font_size    

            # Title
            if 'title' not in config:
              config['title'] = dict()
            title = config['title']
            if 'fontSize' not in title:
              title['fontSize'] = title_font_size

            # Marks
            if 'mark' not in config:
              config['mark'] = dict()
            mark = config['mark']
            if 'fontSize' not in mark:
              mark['fontSize'] = mark_font_size

            # Mark tooltips
            if tooltip and 'tooltip' not in mark:
              mark['tooltip'] = dict(content="encoding")

            return existingTheme
            
        return patch_theme

    # We can only do this once per session
    if theme_sentinel not in alt.themes.names():
      for name in alt.themes.names():
        alt.themes.register(name, make_theme(name))
      
      # register a sentinel theme so we only do this once
      alt.themes.register(theme_sentinel, make_theme('default'))
      alt.themes.enable('default')

  except Exception:
    pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass

# interactivity
if interactivity:
  from IPython.core.interactiveshell import InteractiveShell
  InteractiveShell.ast_node_interactivity = interactivity

# NOTE: the kernel_deps code is repeated in the cleanup.py file
# (we can't easily share this code b/c of the way it is run).
# If you edit this code also edit the same code in cleanup.py!

# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
run_path = 'L1VzZXJzL3Nrb2pha3UtYWRtaW4vRG9jdW1lbnRzL3Byb2plY3RzL2FwcGxpZWQtc29mdC1jb21wL2RvY3MvbGVjdHVyZS1ub3RlL20wMy10ZXh0'
if run_path:
  # hex-decode the path
  run_path = base64.b64decode(run_path.encode("utf-8")).decode("utf-8")
  os.chdir(run_path)

# reset state
%reset

# shiny
# Checking for shiny by using False directly because we're after the %reset. We don't want
# to set a variable that stays in global scope.
if False:
  try:
    import htmltools as _htmltools
    import ast as _ast

    _htmltools.html_dependency_render_mode = "json"

    # This decorator will be added to all function definitions
    def _display_if_has_repr_html(x):
      try:
        # IPython 7.14 preferred import
        from IPython.display import display, HTML
      except:
        from IPython.core.display import display, HTML

      if hasattr(x, '_repr_html_'):
        display(HTML(x._repr_html_()))
      return x

    # ideally we would undo the call to ast_transformers.append
    # at the end of this block whenver an error occurs, we do 
    # this for now as it will only be a problem if the user 
    # switches from shiny to not-shiny mode (and even then likely
    # won't matter)
    import builtins
    builtins._display_if_has_repr_html = _display_if_has_repr_html

    class _FunctionDefReprHtml(_ast.NodeTransformer):
      def visit_FunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

      def visit_AsyncFunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

    ip = get_ipython()
    ip.ast_transformers.append(_FunctionDefReprHtml())

  except:
    pass

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v

  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define
globals()["__spec__"] = None

{"/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/importlib/_bootstrap.py": 1703345986.9647622, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/importlib/_bootstrap_external.py": 1703345986.9687936, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/codecs.py": 1703345984.5304527, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/encodings/aliases.py": 1703345985.7136338, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/encodings/__init__.py": 1703345985.707323, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/encodings/utf_8.py": 1703345986.387034, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/abc.py": 1703345984.4232924, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/io.py": 1703345984.6479, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3.10/stat.py": 1703345984.8209362, "/Users/skojaku-admin/miniforge3/envs/applsoftcomp/lib/python3

/var/folders/j7/9dgqq5g53vnbsbmvh2yqtckr0000gr/T/ipykernel_32107/3548943389.py:25: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats(fig_format)


In [2]:
instruction = "Summarize this abstract"
data = """
We develop a graph neural network for predicting protein-protein interactions
from sequence data. Our model uses attention mechanisms to identify functionally
important amino acid subsequences. We achieve 89% accuracy on benchmark datasets,
outperforming previous methods by 7%. The model also provides interpretable
attention weights showing which protein regions drive predictions.
"""

prompt = f"{instruction}. {data}"

In [3]:
#| code-fold: true

import ollama

params_llm = {"model": "gemma3:270m", "options": {"temperature": 0.3}}

response = ollama.generate(prompt=prompt, **params_llm)
print(response.response)

This abstract describes a graph neural network (GNN) for predicting protein-protein interactions. The model uses attention mechanisms to identify functionally important amino acid subsequences. It achieves 89% accuracy on benchmark datasets and provides interpretable attention weights, indicating its effectiveness.



In [4]:
output_format = """Provide the summary in exactly 2 sentences:
- First sentence: What problem and method
- Second sentence: Key result with numbers"""

prompt_with_format = f"""{instruction}. {data}. {output_format}"""

In [5]:
# New example for persona demonstration
instruction = "Help the customer reconnect to the service by providing troubleshooting instructions."
data = "Customer: I cannot see any webpage. Need help ASAP!"
output_format = "Keep the response concise and polite. Provide a clear resolution in 2-3 sentences."

formal_persona = "You are a professional customer support agent who responds formally and ensures clarity and professionalism."

prompt_with_persona = f"""{formal_persona}. {instruction}. {data}. {output_format}"""

In [6]:
#| code-fold: true
print("BASE (no persona):")
print(ollama.generate(prompt=instruction + ". " + data + ". " + output_format, **params_llm).response)
print("\n" + "="*60 + "\n")
print("WITH PERSONA:")
print(ollama.generate(prompt=prompt_with_persona, **params_llm).response)

BASE (no persona):


Please try re-enabling the webpage. If that doesn't work, please let me know and we can explore other options.



WITH PERSONA:


Hello, I understand you cannot see any webpage. Could you please try accessing the website again? I'll do my best to assist you.



In [7]:
context_background = """The customer is extremely frustrated because their internet has been down for three days, and they need it for an important online job interview. They emphasize that 'This is a life-or-death situation for my career!'"""

prompt_with_context = f"""{formal_persona}. {instruction}. {data}. {output_format}. Context: {context_background}"""

In [8]:
#| code-fold: true
print("WITH PERSONA:")
print(ollama.generate(prompt=prompt_with_persona, **params_llm).response)
print("\n" + "="*60 + "\n")
print("WITH PERSONA + CONTEXT (background):")
print(ollama.generate(prompt=prompt_with_context, **params_llm).response)

WITH PERSONA:


Okay, I understand. I'm here to assist you with your webpage issue. Please try to re-enter the page. I'll do my best to find a solution.



WITH PERSONA + CONTEXT (background):


Dear [Customer Name],

I understand your frustration with the internet outage. I apologize for the inconvenience this has caused. To help me troubleshoot this issue, could you please provide me with the exact URL of the webpage you're having trouble seeing? I'll do my best to assist you.



In [9]:
# Context with audience information for non-technical user
context_with_audience_nontech = f"""{context_background} The customer does not know any technical terms like modem, router, networks, etc."""

context_with_audience_tech = f"""{context_background} The customer is Head of IT Infrastructure of our company."""

prompt_with_context_nontech = f"""{formal_persona}. {instruction}. {data}. {output_format}. Context: {context_with_audience_nontech}"""
prompt_with_context_tech = f"""{formal_persona}. {instruction}. {data}. {output_format}. Context: {context_with_audience_tech}"""

In [10]:
#| code-fold: true
print("WITH PERSONA + CONTEXT (background only):")
print(ollama.generate(prompt=prompt_with_context, **params_llm).response)
print("\n" + "="*60 + "\n")
print("WITH PERSONA + CONTEXT (background + non-tech audience):")
print(ollama.generate(prompt=prompt_with_context_nontech, **params_llm).response)
print("\n" + "="*60 + "\n")
print("WITH PERSONA + CONTEXT (background + tech audience):")
print(ollama.generate(prompt=prompt_with_context_tech, **params_llm).response)

WITH PERSONA + CONTEXT (background only):


Dear [Customer Name],

I understand your frustration with your internet connection and the need for this important job interview. I'm here to assist you with troubleshooting. Please provide me with the specific error message or the steps you've already taken to try to connect. I'll do my best to help you get back online as quickly as possible.



WITH PERSONA + CONTEXT (background + non-tech audience):


Thank you for contacting us. I understand your frustration regarding your internet outage. I'm happy to help you troubleshoot this issue. Please provide me with the specific error message you're seeing, and I'll do my best to assist you.



WITH PERSONA + CONTEXT (background + tech audience):


"I understand your frustration, and I apologize for the inconvenience this is causing. We're currently experiencing a significant internet outage, and we're working to resolve this as quickly as possible. We'll be back with you as soon as we have a definitive solution. Please don't hesitate to reach out if you have any further questions."


In [11]:
prompt_template = """
{persona}

{instruction}

{data}

Context: {context}

{output_format}
"""

In [12]:
zero_shot_prompt = """Extract the domain and methods from this abstract:

Abstract: We apply reinforcement learning to optimize traffic flow in urban networks.
Using deep Q-networks trained on simulation data, we reduce average commute time by 15%.

Output format:
Domain: ...
Methods: ...
"""

In [13]:
few_shot_prompt = """Extract the domain and methods from abstracts. Here are examples:

Example 1:
Abstract: We use CRISPR to edit genes in cancer cells, achieving 40% tumor reduction in mice.
Domain: Cancer Biology
Methods: CRISPR gene editing, mouse models

Example 2:
Abstract: We develop a transformer model for predicting solar flares from magnetogram images.
Domain: Solar Physics, Machine Learning
Methods: Transformer neural networks, image analysis

Now extract from this abstract:

Abstract: We apply reinforcement learning to optimize traffic flow in urban networks.
Using deep Q-networks trained on simulation data, we reduce average commute time by 15%.

Domain: ...
Methods: ...
"""

In [14]:
#| code-fold: true
response_zero = ollama.generate(prompt=zero_shot_prompt, **params_llm)
response_few = ollama.generate(prompt=few_shot_prompt, **params_llm)

print("ZERO-SHOT:")
print(response_zero.response)
print("\nFEW-SHOT:")
print(response_few.response)

ZERO-SHOT:
Domain: Urban networks
Methods: Reinforcement Learning

FEW-SHOT:
Here's the extracted domain and methods from the abstract:

*   **Domain:** Science
*   **Methods:** Reinforcement Learning


In [15]:
papers = """
Paper 1: Community detection in static networks using modularity optimization.
Paper 2: Temporal network analysis with sliding windows.
Paper 3: Hierarchical community structure in social networks.
"""

direct_prompt = f"""Based on these paper titles, what research gap exists? Just give the answer, no explanation.

{papers}

Gap: ...
"""

In [16]:
cot_prompt = f"""Based on these paper titles, identify a research gap. Think step by step.

Papers:
{papers}

Think step by step:
1. What does each paper focus on?
2. What topics appear in multiple papers?
3. What combination of topics is missing?
4. What would be a valuable gap to fill?

Final answer: The research gap is...
"""

In [17]:
#| code-fold: true
response_direct = ollama.generate(prompt=direct_prompt, **params_llm)
response_cot = ollama.generate(prompt=cot_prompt, **params_llm)

print("DIRECT PROMPT:")
print(response_direct.response)
print("\nCHAIN-OF-THOUGHT:")
print(response_cot.response)

DIRECT PROMPT:
The gap is in the complexity of the models used for community detection and the types of data used to train them.


CHAIN-OF-THOUGHT:
Here's the breakdown of the research gap identified:

1.  **What does each paper focus on?**
    *   Community detection in static networks using modularity optimization.

2.  **What topics appear in multiple papers?**
    *   Temporal network analysis with sliding windows.

3.  **What combination of topics is missing?**
    *   Hierarchical community structure in social networks.

4.  **What would be a valuable gap to fill?**
    *   The gap is the lack of a comprehensive, integrated study that addresses the intersection of community detection, temporal network analysis, and hierarchical community structure in social networks.


In [18]:
import json
from pydantic import BaseModel

abstract = """
We analyze 10,000 scientific collaborations using network analysis and machine
learning. Our random forest classifier predicts collaboration success with 76%
accuracy. Key factors include prior co-authorship and institutional proximity.
"""

prompt_json = f"""Extract information from this abstract and return ONLY valid JSON:

Abstract: {abstract}

Return this exact structure:
{{
  "n_samples": <number or null>,
  "methods": [<list of methods>],
  "accuracy": <number or null>,
  "domain": "<research field>"
}}

JSON:"""

In [19]:
#| code-fold: true
# Use lower temperature for structured output
params_structured = {"model": "gemma3n:latest", "options": {"temperature": 0}}
response = ollama.generate(prompt=prompt_json, **params_structured)

try:
    data = json.loads(response.response)
    print("Extracted data:")
    print(json.dumps(data, indent=2))
except json.JSONDecodeError:
    print("Failed to parse JSON. Raw output:")
    print(response.response)

Failed to parse JSON. Raw output:
```json
{
 "n_samples": 10000,
 "methods": ["network analysis", "machine learning", "random forest"],
 "accuracy": 76,
 "domain": "scientific collaborations"
}
```


In [20]:
from pydantic import BaseModel

class PaperMetadata(BaseModel):
    domain: str
    methods: list[str]
    n_samples: int | None
    accuracy: float | None

json_schema = PaperMetadata.model_json_schema()

In [21]:
prompt_schema = f"""Extract information from this abstract:

Abstract: {abstract}"""

In [22]:
#| code-fold: true
response = ollama.generate(prompt=prompt_schema, format=json_schema, **params_structured)

try:
    data = json.loads(response.response)
    metadata = PaperMetadata(**data)
    print("Extracted and validated data:")
    print(json.dumps(data, indent=2))
except (json.JSONDecodeError, ValueError) as e:
    print(f"Error: {e}")
    print("Raw output:", response.response)

Extracted and validated data:
{
  "domain": "Scientific Collaborations",
  "methods": [
    "Network Analysis",
    "Machine Learning",
    "Random Forest Classifier"
  ],
  "n_samples": 10000,
  "accuracy": 76.0
}


In [23]:
bad_prompt = """Summarize the main findings from the 2023 paper by Johnson et al.
on quantum community detection in biological networks."""

In [24]:
good_prompt = """I'm looking for a 2023 paper by Johnson et al. on quantum
community detection in biological networks.

If you know this paper, summarize its main findings.
If you're not certain this paper exists, say "I cannot verify this paper exists"
and do NOT make up details.

Response:"""

In [25]:
#| code-fold: true
response_bad = ollama.generate(prompt=bad_prompt, **params_llm)
response_good = ollama.generate(prompt=good_prompt, **params_llm)

print("BAD PROMPT (encourages hallucination):")
print(response_bad.response)
print("\nGOOD PROMPT (allows uncertainty):")
print(response_good.response)

BAD PROMPT (encourages hallucination):
The 2023 paper by Johnson et al. on quantum community detection in biological networks, titled "Quantum Community Detection in Biological Networks," investigated the effectiveness of quantum-based methods for detecting and characterizing biological networks. The research focused on the challenges and opportunities associated with quantum detection, including the need for specialized hardware and software, the potential for low sensitivity and high error rates, and the need for robust and reliable quantum algorithms. The paper highlighted the importance of understanding the underlying mechanisms of quantum community detection and the potential for developing quantum-enhanced methods to overcome these limitations.

GOOD PROMPT (allows uncertainty):
I cannot verify the existence of this paper.



In [26]:
from collections import Counter

prompt_consistency = """Three papers study network robustness:
- Paper A: Targeted attacks are most damaging
- Paper B: Random failures rarely cause collapse
- Paper C: Hub nodes are critical for robustness

What is the research consensus on network robustness? Give a one-sentence answer.
"""

In [27]:
#| code-fold: true
# Use higher temperature for diversity
params_creative = {"model": "gemma3n:latest", "options": {"temperature": 0.7}}

# Generate 5 responses
responses = []
for i in range(5):
    response = ollama.generate(prompt=prompt_consistency, **params_creative)
    responses.append(response.response.strip())
    print(f"Response {i+1}: {responses[-1]}\n")

# In practice, you'd programmatically identify the most common theme
print("The most consistent theme across responses would be selected.")

Response 1: The research consensus on network robustness is that it's a complex issue influenced by both targeted attacks and random failures, with the vulnerability of critical hub nodes playing a significant role in overall network resilience.



Response 2: The research consensus on network robustness is that it's a multifaceted issue involving vulnerabilities to both targeted attacks and random failures, with the criticality of hub nodes playing a significant role in overall network resilience.



Response 3: The research consensus on network robustness is that it's a complex issue influenced by both targeted attacks and random failures, with the vulnerability of critical hub nodes playing a significant role in overall network resilience.



Response 4: The research consensus on network robustness is that it's a complex issue influenced by both targeted attacks and random failures, with the vulnerability of critical hub nodes playing a significant role in overall network stability.



Response 5: The research consensus on network robustness is that it's a complex issue influenced by factors like targeted attacks, random failures, and the importance of critical nodes, suggesting a multifaceted approach is needed to understand and improve network resilience.

The most consistent theme across responses would be selected.


In [28]:
# Use low temperature for consistency
params_classifier = {"model": "gemma3n:latest", "options": {"temperature": 0}}

def classify_paper(abstract):
    """Classify a paper's methodology type."""
    prompt = """Classify research papers by methodology type.

Categories:
- Experimental: Lab/field experiments, data collection
- Theoretical: Mathematical models, proofs, frameworks
- Computational: Simulations, algorithms, data analysis
- Review: Literature reviews, meta-analyses, surveys

Examples:

Abstract: We prove a lower bound on the complexity of community detection algorithms.
Classification: Theoretical
Reasoning: Focuses on mathematical proof.

Abstract: We conduct surveys with 500 participants to study social network formation.
Classification: Experimental
Reasoning: Original data collection through experiments.

Abstract: We review 150 papers on graph neural networks and identify future directions.
Classification: Review
Reasoning: Surveys existing literature.

Now classify:

Abstract: {abstract}

Think step by step:
1. What is the primary activity? (proving, measuring, simulating, surveying?)
2. Which category fits best?

Classification: ...
Reasoning: ...
"""
    response = ollama.generate(prompt=prompt.format(abstract=abstract), **params_classifier)
    return response.response

# Test with different paper types
test_abstracts = [
    "We develop a graph neural network that predicts protein folding with 85% accuracy.",
    "We mathematically prove that scale-free networks are robust to random failures.",
    "We survey 200 papers on community detection and identify 5 major approaches.",
]

for abstract in test_abstracts:
    print(f"Abstract: {abstract}")
    print(classify_paper(abstract))
    print("-" * 80)

Abstract: We develop a graph neural network that predicts protein folding with 85% accuracy.


Classification: Computational
Reasoning: The abstract explicitly mentions "develop a graph neural network" and "predicts protein folding with 85% accuracy." This indicates the use of simulations and algorithms to solve a problem, which falls under computational methodology.

--------------------------------------------------------------------------------
Abstract: We mathematically prove that scale-free networks are robust to random failures.


Classification: Theoretical
Reasoning: The abstract explicitly states a mathematical proof. This aligns directly with the definition of a theoretical research paper, which focuses on mathematical models and proofs.

--------------------------------------------------------------------------------
Abstract: We survey 200 papers on community detection and identify 5 major approaches.


Classification: Review
Reasoning: The abstract explicitly states a survey of existing papers (200 papers) and identification of approaches. This aligns directly with the definition of a literature review.

--------------------------------------------------------------------------------


In [29]:
# Use a more open-ended prompt to show temperature differences
prompt_temp = "Suggest a creative name for a research project about network analysis:"

print("Temperature = 0 (deterministic):")
for i in range(3):
    response = ollama.generate(
        prompt=prompt_temp,
        model="gemma3n:latest",
        options={"temperature": 0}
    )
    print(f"  {i+1}. {response.response.strip()}")

print("\nTemperature = 1.0 (creative):")
for i in range(3):
    response = ollama.generate(
        prompt=prompt_temp,
        model="gemma3n:latest",
        options={"temperature": 1.0}
    )
    print(f"  {i+1}. {response.response.strip()}")

Temperature = 0 (deterministic):


  1. Okay, here are some creative names for a network analysis research project, categorized by the feeling they evoke. I've tried to include a variety of styles – some are more technical, some are more evocative, and some are a bit playful.

**I. Technical & Precise:**

*   **Nexus Dynamics:** (Highlights connections and change)
*   **Interwoven Insights:** (Emphasizes the complexity of networks)
*   **Graphic Intelligence:** (Combines visual representation with AI/analysis)
*   **Network Topology & Resilience:** (Focuses on structural properties and robustness)
*   **Algorithmic Interconnectivity:** (Highlights the computational aspect)
*   **NodeFlow:** (Simple, memorable, and suggests data movement)
*   **Structural Informatics:** (A more academic/formal term)
*   **Networked Systems Analysis (NSA):** (A clear, concise acronym)

**II. Evocative & Intriguing:**

*   **The Web Within:** (Suggests hidden patterns and underlying structures)
*   **Echoes of Connection:** (Implies the ri

  2. Okay, here are some creative names for a network analysis research project, categorized by the feeling they evoke. I've tried to include a variety of styles – some are more technical, some are more evocative, and some are a bit playful.

**I. Technical & Precise:**

*   **Nexus Dynamics:** (Highlights connections and change)
*   **Interwoven Insights:** (Emphasizes the complexity of networks)
*   **Graphic Intelligence:** (Combines visual representation with AI/analysis)
*   **Network Topology & Resilience:** (Focuses on structural properties and robustness)
*   **Algorithmic Interconnectivity:** (Highlights the computational aspect)
*   **NodeFlow:** (Simple, memorable, and suggests data movement)
*   **Structural Informatics:** (A more academic/formal term)
*   **Networked Systems Analysis (NSA):** (A clear, concise acronym)

**II. Evocative & Intriguing:**

*   **The Web Within:** (Suggests hidden patterns and underlying structures)
*   **Echoes of Connection:** (Implies the ri

  3. Okay, here are some creative names for a network analysis research project, categorized by the feeling they evoke. I've tried to include a variety of styles – some are more technical, some are more evocative, and some are a bit playful.

**I. Technical & Precise:**

*   **Nexus Dynamics:** (Highlights connections and change)
*   **Interwoven Insights:** (Emphasizes the complexity of networks)
*   **Graphic Intelligence:** (Combines visual representation with AI/analysis)
*   **Network Topology & Resilience:** (Focuses on structural properties and robustness)
*   **Algorithmic Interconnectivity:** (Highlights the computational aspect)
*   **NodeFlow:** (Simple, memorable, and suggests data movement)
*   **Structural Informatics:** (A more academic/formal term)
*   **Networked Systems Analysis (NSA):** (A clear, concise acronym)

**II. Evocative & Intriguing:**

*   **The Web Within:** (Suggests hidden patterns and underlying structures)
*   **Echoes of Connection:** (Implies the ri

  1. Okay, here are some creative names for a network analysis research project, categorized by the feeling they evoke. I've tried to include a mix of serious, intriguing, and slightly playful options.

**Serious & Academic:**

*   **Nexus Dynamics:**  (Suggests interconnectedness and change)
*   **The Interwoven Landscape:** (Evokes complexity and exploration)
*   **Network Topology & Resilience:** (Focuses on structural properties and robustness)
*   **Algorithmic Entanglements:** (Highlights the computational aspect)
*   **Structural Insights: A Network Perspective:** (Clear, concise, and informative)
*   **The Connected Ecosystem:** (Broad, applicable to many domains)
*   **Complex Systems & Network Behavior:** (Academic and comprehensive)

**Intriguing & Evocative:**

*   **The Hidden Threads:** (Suggests uncovering unseen relationships)
*   **Echoes in the Network:** (Implies patterns and reverberations)
*   **The Flow of Influence:** (Focuses on the direction and impact of conne

  2. ## Creative Names for a Network Analysis Research Project:

Here's a breakdown of names, categorized by their focus and style, with explanations of why they work:

**I. Evocative & Poetic:**

*   **The Weave:**  Simple, memorable, and suggests interconnectedness.
*   **Echoes in the Network:**  Hints at information flow and reverberations within the network.
*   **The Invisible Threads:**  Emphasizes the hidden structures and relationships.
*   **Nexus Point:**  Highlights central nodes and critical connections.
*   **Resonance:**  Suggests how information and influence propagate through the network.
*   **The Labyrinth of Links:**  A bit more dramatic, implying complexity and exploration.

**II.  Technical & Intriguing:**

*   **Network Topology & Dynamics: A [Specific Domain] Perspective:** (e.g., "A Social Media Perspective") -  Clear, informative, and academic.
*   **Algorithmic Cartography of [Network Type]:**  Uses technical language to sound sophisticated.
*   **Complex Sys

  3. Okay, here are some creative names for a network analysis research project, categorized by their feel (technical, evocative, intriguing, etc.). I've tried to include a variety to suit different project focuses.

**I. Technical & Precise:**

*   **Nexus Dynamics:**  (Highlights connections and change)
*   **Graphformatics Insights:** (Combines graph theory and data science)
*   **Network Topology & Resilience:** (Good for projects focused on robustness)
*   **Algorithmic Interconnectivity:** (Emphasizes the algorithms used)
*   **Structural Data Mining:** (Focuses on extracting information from network structures)
*   **Connected Systems Analysis:** (Broad, but professional)
*   **Network Propagation & Diffusion:** (If the project deals with how information spreads)

**II. Evocative & Intriguing:**

*   **The Web of Influence:** (Suggests power dynamics and impact)
*   **Hidden Currents:** (Implies uncovering unseen connections)
*   **The Interwoven:** (Poetic, suggests complexity 